In [1]:
import healpy as hp
import itertools
import jax.numpy as jnp
from jax import vmap, grad
from response import pairwise_monopole, pairwise_dipole, vvec,dir,pulsar_star_pair_monopole, pulsar_star_pair_dipole
from jax import config
# config.update("jax_enable_x64", True)
import numpy as np
# plot settings
import matplotlib
import matplotlib.pyplot as plt
np.set_printoptions(precision=4)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# set our test vectors

theta1 = np.pi/2

theta2 = np.pi/4

nvec = dir(theta1,0)

xvec = dir(theta2,0)

nside = 64
npix = hp.nside2npix(nside)
pix = np.arange(npix)
pvec_array = np.array(hp.pix2vec(nside,pix)).T

mono = pulsar_star_pair_monopole(nvec,xvec,pvec_array,nside)

print(mono)

[ 1.9238e-16 -6.3393e-09 -2.9948e+00]


In [3]:
# analytic from GT

def monopole_GT(nvec,xvec):
    const = 16*np.pi/3
    ndotx = jnp.sum(nvec*xvec)
    y = 0.5*(1-ndotx)
    fy = (2*y - 2*y**2 + 3*y**2*jnp.log(y))
    mid = (ndotx*nvec -  xvec)/(1 - ndotx**2)
    return const * mid * fy

print(monopole_GT(nvec,xvec))

print("Ratio Monopole = ",monopole_GT(nvec,xvec) / mono)

[ 0.     0.    -2.995]
Ratio Monopole =  [ 0.     -0.      1.0001]


In [4]:
theta0 = np.deg2rad(48) 
phi0 = np.deg2rad(264)
vvec = dir(theta0,phi0)
pi = np.pi

nside = 64

theta1 = np.pi/2

theta2 = np.pi/6

nvec = dir(theta1,0)

xvec = dir(theta2,0)

dipole = pulsar_star_pair_dipole(nvec,xvec,pvec_array,nside)

print("Numerical dipole = ",dipole)

def dipole_GT(nvec,xvec,vvec=vvec):
    ndotx = jnp.sum(nvec*xvec)
    y = 0.5*(1-ndotx)
    ndotv = jnp.sum(nvec*vvec)
    xdotv = jnp.sum(xvec*vvec)

    def get_c1():
        den = y-1
        num_t1 =1/6 *  pi*( (y*(12*(y-1)*y-5)-4)*(ndotv*(2*y-1) + xdotv) + 36*ndotv*jnp.sqrt(2-2*y)*y**3)
        num_t2 =1/6 * (34*y + 8)*pi*ndotv*jnp.sqrt(2-2*y)*y #There was a -34 instead of +34
        num_t3 = 8*pi*y**2*jnp.log(y)*( ndotv*(2*y + jnp.sqrt(2-2*y) -1) + xdotv)
        res = (num_t1-num_t2-num_t3)/den
        return res

    def get_c2():
        den_1 = (y-1)**2
        den_2 = 6*y*(1-y)
        num_1 = 2*pi*y*(ndotv+xdotv)*(ndotv*(2*y-1) + xdotv)*jnp.log(y)
        num_2 = pi*(ndotv*(2*y-1) + xdotv) * (ndotv*(8*y**2 + 6*y - 1) +12*xdotv*y**2 + xdotv )
        res = num_1/den_1 + num_2/den_2
        return res

    c1 = get_c1()
    c2 = get_c2()

    def get_a1():
        num = c1*(ndotv**2 - 1) + c2*(xdotv - ndotv*ndotx)
        den = ndotv**2 - 2*ndotv*xdotv*ndotx + ndotx**2 + xdotv**2 - 1
        return num/den

    def get_a2():
        num = c1*(xdotv - ndotv*ndotx) + c2*(ndotx**2 - 1)
        den = ndotv**2 - 2*ndotv*xdotv*ndotx + ndotx**2 + xdotv**2 - 1
        return num/den        

    a1 = get_a1()

    a2 = get_a2()

    K_i = (a1 * (xvec - ndotx*nvec)) + a2 * (vvec - ndotv*nvec)

    return K_i

print("Analytical dipole = ",dipole_GT(nvec,xvec))

print("Ratio = ",dipole_GT(nvec,xvec) / dipole)

Numerical dipole =  [ 2.3170e-17 -9.4997e-01 -5.4799e-01]
Analytical dipole =  [0.     0.2445 0.8182]
Ratio =  [ 0.     -0.2574 -1.4931]


In [5]:
print(vvec)

from response import vvec as vcmb

print(vcmb)

[-0.0777 -0.7391  0.6691]
[-0.0777 -0.7391  0.6691]
